In [1]:
import json
import argparse
from pathlib import Path
import os.path
import dtlpy as dataloop
import subprocess

In [7]:
data_info = {
    'project': "Car Condition Classification",
    'dataset': "360 Portrait",
    'email': "ashley.lawrencehuizenga@coxautoinc.com",
    'out_dir': "dataloop/",
    'password': ""
}

In [3]:
dataloop.login_m2m(email=data_info['email'], password=data_info['password'])

True

In [8]:
proj = dataloop.projects.get(data_info['project'])

In [9]:
dataset = proj.datasets.get(data_info['dataset'])

In [10]:
dataset.download(local_path=data_info["out_dir"])
dataset.download_annotations(local_path=data_info["out_dir"])

Download Items: 100%|██████████| 1/1 [00:00<00:00,  1.90it/s]


'dataloop/'